In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [69]:
RAW_PATH = r"/kaggle/input/adnisnp/ADNI_qc_pruned_target.raw"      
OUT_EMBED_CSV = r"genotype_embeddings_128.csv"

In [3]:
IID_col = 'IID'
#scale_features = True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
df = pd.read_csv(RAW_PATH, sep='\s+')

/tmp/ipykernel_36/3037716494.py:1: DtypeWarning: Columns (0,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_PATH, sep='\s+')


In [6]:
print("IID" in df.columns)

True


In [6]:
meta_cols = ['FID','IID','PAT','MAT','SEX','PHENOTYPE']
iid = df[IID_col].astype(str).copy()

In [7]:
g_df = df.drop(columns=[c for c in meta_cols if c in df.columns])

In [8]:
g_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,rs3094315_C,rs12562034_A,rs12124819_G,rs3748597_T,rs28705211_C,rs13303118_G,rs9777703_C,rs3934834_T,rs3737728_T,rs6687776_T,...,MitoA15759G_G,MitoC15834T_T,MitoC15905T_T,MitoA15925G_G,MitoG16130A_A,MitoC16149T_T,MitoA16163G_G,MitoA16164G_G,MitoC16184A_C,MitoG16392A_A
0,0.0,1.0,0.0,0.0,1.0,1.0,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.0,0.0,2.0,1.0,0.0,2.0,0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
582,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
583,0.0,0.0,1.0,0.0,1.0,1.0,0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
584,0.0,0.0,2.0,0.0,0.0,0.0,0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.impute import SimpleImputer

In [10]:
g_df = g_df.apply(pd.to_numeric, errors='coerce')

In [11]:
imp = SimpleImputer(strategy="mean")
g_im = pd.DataFrame(
    imp.fit_transform(g_df),
    columns=g_df.columns,   
    index=g_df.index
)

In [12]:
g_im

,rs3094315_C,rs12562034_A,rs12124819_G,rs3748597_T,rs28705211_C,rs13303118_G,rs9777703_C,rs3934834_T,rs3737728_T,rs6687776_T,...,MitoA15759G_G,MitoC15834T_T,MitoC15905T_T,MitoA15925G_G,MitoG16130A_A,MitoC16149T_T,MitoA16163G_G,MitoA16164G_G,MitoC16184A_C,MitoG16392A_A
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094643,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
582,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.000000,0.0
583,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
584,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [13]:
selector = VarianceThreshold(threshold=0.0)  
selector.fit(g_im)


VarianceThreshold()

In [14]:
keep_cols = g_im.columns[selector.get_support()]
geno_filtered = g_im[keep_cols]

In [15]:
print("Original SNPs:", g_df.shape[1])
print("After variance filter:", g_im.shape[1])

Original SNPs: 114923
After variance filter: 114923


In [16]:
print("Total NaNs in geno_df ->", g_df.isna().sum().sum())
print("Total NaNs in geno_im ->", g_im.isna().sum().sum())

Total NaNs in geno_df -> 376650
Total NaNs in geno_im -> 0


In [24]:
g_im

,rs3094315_C,rs12562034_A,rs12124819_G,rs3748597_T,rs28705211_C,rs13303118_G,rs9777703_C,rs3934834_T,rs3737728_T,rs6687776_T,...,MitoA15759G_G,MitoC15834T_T,MitoC15905T_T,MitoA15925G_G,MitoG16130A_A,MitoC16149T_T,MitoA16163G_G,MitoA16164G_G,MitoC16184A_C,MitoG16392A_A
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094643,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
582,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.000000,0.0
583,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
584,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [23]:
geno_filtered

,rs3094315_C,rs12562034_A,rs12124819_G,rs3748597_T,rs28705211_C,rs13303118_G,rs9777703_C,rs3934834_T,rs3737728_T,rs6687776_T,...,MitoA15759G_G,MitoC15834T_T,MitoC15905T_T,MitoA15925G_G,MitoG16130A_A,MitoC16149T_T,MitoA16163G_G,MitoA16164G_G,MitoC16184A_C,MitoG16392A_A
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094643,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.0,0.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
582,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.000000,0.0
583,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
584,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [17]:
X = g_im.values.astype(float)

In [18]:
X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.]])

In [19]:
X.shape

(586, 114923)

In [82]:
class GenotypeAE(nn.Module):
    def __init__(self, input_dim, latent_dim=128, p_drop=0.05):
        super().__init__()
        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(p_drop),
            nn.Linear(2048, 1028),
            nn.BatchNorm1d(1028),
            nn.ReLU(),
            nn.Linear(1028, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(p_drop),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, input_dim)
            # no activation — we'll use MSELoss
        )

    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)
        return xhat, z


In [83]:
Latent_dim = 96
LR = 0.001
Epoch = 30
BATCH_SIZE = 32

In [84]:
X_tensor = torch.from_numpy(X)
X_tensor = X_tensor.to(torch.float32)
train_loader = DataLoader(TensorDataset(X_tensor, X_tensor), batch_size=BATCH_SIZE, shuffle=True)
full_loader = DataLoader(TensorDataset(X_tensor, X_tensor), batch_size=BATCH_SIZE, shuffle=False)

In [85]:
input_dim = X.shape[1]
model = GenotypeAE(input_dim=input_dim, latent_dim=Latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
loss_fn = nn.MSELoss()

In [86]:
for epoch in range(1, Epoch+1):
    model.train()
    epoch_loss = 0.0
    for xb, _ in train_loader:
        xb = xb.to(device)
        noise = 0.01 * torch.randn_like(xb)   
        xb_noisy = xb + noise
        optimizer.zero_grad()
        xhat, _ = model(xb_noisy)
        loss = loss_fn(xhat, xb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * xb.size(0)
    epoch_loss /= X.shape[0]
    if epoch % 5 == 0 or epoch==1:
        print(f"Epoch {epoch}/{Epoch} — Train MSE: {epoch_loss:.6f}")

Epoch 1/30 — Train MSE: 0.449664
Epoch 5/30 — Train MSE: 0.330180
Epoch 10/30 — Train MSE: 0.329730
Epoch 15/30 — Train MSE: 0.328769
Epoch 20/30 — Train MSE: 0.326498
Epoch 25/30 — Train MSE: 0.325823
Epoch 30/30 — Train MSE: 0.324845


In [67]:
model.eval()
embeddings = []
with torch.no_grad():
    for xb, _ in full_loader:
        xb = xb.to(device)
        _, z = model(xb)
        embeddings.append(z.cpu().numpy())
embeddings = np.vstack(embeddings)   
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (586, 128)


In [70]:
emb_df = pd.DataFrame(embeddings, index=iid, columns=[f"g_emb_{i}" for i in range(1, embeddings.shape[1]+1)])
emb_df.index.name = IID_col
emb_df.reset_index(inplace=True)
emb_df.to_csv(OUT_EMBED_CSV, index=False)
print("Saved genotype embeddings to:", OUT_EMBED_CSV)

Saved genotype embeddings to: genotype_embeddings_128.csv


In [73]:
import numpy as np
from sklearn.decomposition import PCA

# PCA baseline with same latent dim
latent_dim = 64  # or 128
pca = PCA(n_components=latent_dim, svd_solver='randomized')
X_pca = pca.fit_transform(X)
Xhat_pca = pca.inverse_transform(X_pca)
mse_pca = np.mean((X - Xhat_pca)**2)
print("PCA MSE:", mse_pca)


PCA MSE: 0.2704885820372584


In [74]:
print("PCA explained variance (64 comps):", pca.explained_variance_ratio_.sum())

PCA explained variance (64 comps): 0.1554673489893944


In [88]:
for k in [64, 128, 256, 512,1024,2048]:
    pca = PCA(n_components=k, svd_solver='randomized')
    X_pca = pca.fit_transform(X)
    print(k, pca.explained_variance_ratio_.sum())


64 0.15555654505292024
128 0.2666622482157711
256 0.4827999405400144
512 0.8917181263423389


ValueError: n_components=1024 must be between 1 and min(n_samples, n_features)=586 with svd_solver='randomized'